In [1]:
import cv2
import numpy as np

In [2]:
def merge_boxes(box1, box2):
  """
  두 박스를 하나의 박스로 합칩니다.

  Args:
    box1: 첫 번째 박스
    box2: 두 번째 박스

  Returns:
    합쳐진 박스
  """

  b1_x, b1_y, b1_w, b1_h = box1
  b2_x, b2_y, b2_w, b2_h = box2
  
  return (
    min(b1_x, b2_x),
    min(b1_y, b2_y),
    max(b1_x + b1_w, b2_x + b2_w),
    max(b1_y + b1_h, b2_y + b2_h),
  )


In [3]:
def is_intersecting(box1, box2, min_distance, log_view: bool = False):
  """
  두 박스가 겹쳐지는지 확인합니다.

  Args:
    box1: 첫 번째 박스
    box2: 두 번째 박스

  Returns:
    True if 두 박스가 겹치면, False otherwise
  """

  b1_x, b1_y, b1_w, b1_h = box1
  b2_x, b2_y, b2_w, b2_h = box2

  status = (
    b1_x <= (b2_x + b2_w) + min_distance and
    b2_x <= (b1_x + b1_w) + min_distance and
    b1_y <= (b2_y + b2_h) + min_distance and
    b2_y <= (b1_y + b1_h) + min_distance
  )

  if log_view:
    print(box1, box2, status)

  return status

In [4]:
def group_boxes(boxes, min_distance):
  """
  겹쳐지거나 가까운 박스들을 그룹으로 묶습니다.

  Args:
    boxes: 박스 리스트
    min_distance: 박스간의 최소 거리

  Returns:
    그룹화된 박스 리스트
  """
  
  # 반복적으로 겹쳐지거나 가까운 박스들을 병합합니다.
  group_boxse = [] # 그룹화된 박스를 저장합니다.

  for i in range(1, len(boxes)):
    if is_intersecting(boxes[i-1], boxes[i], min_distance, True): # 두 박스가 겹쳐지는지 확인합니다.
      
      group_box = merge_boxes(boxes[i-0], boxes[i]) # 겹쳐지는 박스를 하나로 합칩니다.
      group_boxse.append(group_box) # 그룹화된 박스 리스트에 추가합니다.

  return group_boxse

In [5]:
# 이미지 불러오기
image = cv2.imread("../dataset_dir/13.한국어글자체/01.손글씨/01_handwriting_sentence_images/1_sentence/00000002.png")

# 컨투어 찾기 전 이미지 전처리
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
gray = cv2.bitwise_not(gray) # 객체보다 배경이 밝은 경우 이미지 반전

# threshold 등 다양한 전처리 시도 -> 객체와 배경을 가장 잘 분리하는 전처리 사용
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

# 윤곽선 찾기 및 박스 추출
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
boxes = [cv2.boundingRect(contour) for contour in contours]

# 최소 거리 설정
min_distance = 2000

# 겹쳐지거나 가까운 박스 그룹화
grouped_boxes = group_boxes(boxes, min_distance)

# 결과 이미지 출력
for group in grouped_boxes:
  x, y, w, h = group
  cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# 결과 이미지 출력
cv2.imwrite(f'./image/asdf.jpg', image)

(3492, 143, 42, 23) (3512, 136, 14, 10) True
(3512, 136, 14, 10) (975, 131, 36, 16) False
(975, 131, 36, 16) (3666, 129, 36, 31) False
(3666, 129, 36, 31) (3660, 115, 36, 16) True
(3660, 115, 36, 16) (1335, 113, 58, 32) False
(1335, 113, 58, 32) (3217, 112, 5, 5) True
(3217, 112, 5, 5) (3468, 111, 63, 22) True
(3468, 111, 63, 22) (2719, 107, 42, 27) True
(2719, 107, 42, 27) (2577, 104, 47, 23) True
(2577, 104, 47, 23) (947, 104, 58, 26) True
(947, 104, 58, 26) (2279, 103, 66, 11) True
(2279, 103, 66, 11) (2603, 101, 13, 6) True
(2603, 101, 13, 6) (1835, 98, 59, 38) True
(1835, 98, 59, 38) (1125, 97, 37, 50) True
(1125, 97, 37, 50) (1549, 96, 41, 33) True
(1549, 96, 41, 33) (1970, 91, 50, 35) True
(1970, 91, 50, 35) (488, 90, 60, 35) True
(488, 90, 60, 35) (247, 90, 53, 50) True
(247, 90, 53, 50) (1319, 89, 61, 13) True
(1319, 89, 61, 13) (3376, 84, 60, 75) True
(3376, 84, 60, 75) (2046, 81, 52, 47) True
(2046, 81, 52, 47) (3357, 75, 5, 5) True
(3357, 75, 5, 5) (3363, 73, 48, 46) True
(

True

In [6]:
grouped_boxes

[(3512, 136, 3526, 146),
 (3660, 115, 3696, 131),
 (3217, 112, 3222, 117),
 (3468, 111, 3531, 133),
 (2719, 107, 2761, 134),
 (2577, 104, 2624, 127),
 (947, 104, 1005, 130),
 (2279, 103, 2345, 114),
 (2603, 101, 2616, 107),
 (1835, 98, 1894, 136),
 (1125, 97, 1162, 147),
 (1549, 96, 1590, 129),
 (1970, 91, 2020, 126),
 (488, 90, 548, 125),
 (247, 90, 300, 140),
 (1319, 89, 1380, 102),
 (3376, 84, 3436, 159),
 (2046, 81, 2098, 128),
 (3357, 75, 3362, 80),
 (3363, 73, 3411, 119),
 (2367, 72, 2431, 126),
 (3091, 70, 3127, 107),
 (3217, 68, 3255, 95),
 (3470, 67, 3539, 116),
 (2858, 65, 2876, 73),
 (2559, 64, 2617, 91),
 (3226, 61, 3284, 141),
 (3140, 57, 3179, 135),
 (1620, 57, 1658, 112),
 (2282, 56, 2318, 90),
 (3653, 54, 3684, 110),
 (565, 54, 614, 113),
 (2724, 53, 2754, 97),
 (1674, 53, 1698, 134),
 (618, 53, 644, 116),
 (1201, 52, 1256, 142),
 (345, 52, 377, 99),
 (1324, 51, 1372, 86),
 (3093, 50, 3118, 63),
 (2858, 50, 2909, 145),
 (55, 50, 94, 105),
 (3234, 47, 3256, 75),
 (2801, 